In [1]:
from google.colab import drive 
drive.mount('/content/gdrive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# model.py

import torch
import torch.nn as nn

from torchvision import models

class MRNet(nn.Module):
    def __init__(self):
        super().__init__()
        #self.model = models.alexnet(pretrained=True)
        self.model = models.alexnet(pretrained=True)
        #self.model = models.googlenet(pretrained=True)
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(256, 1)

    def forward(self, x):
        x = torch.squeeze(x, dim=0) # only batch size 1 supported
        x = self.model.features(x)
        x = self.gap(x).view(x.size(0), -1)
        x = torch.max(x, 0, keepdim=True)[0]
        x = self.classifier(x)
        return x

In [0]:
# loader.py

import numpy as np
import os
import pickle
import torch
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
import PIL

from torch.autograd import Variable

INPUT_DIM = 224
MAX_PIXEL_VAL = 255
MEAN = 58.09
STDDEV = 49.73

class Dataset(data.Dataset):
    def __init__(self, datadirs, diagnosis, use_gpu):
        super().__init__()
        self.use_gpu = use_gpu
        
        label_dict = {}
        self.paths = []
        print(datadirs)

        for i, line in enumerate(open('metadata.csv').readlines()):
            if i == 0:
                continue
            line = line.strip().split(',')
            path = line[10]
            label = line[2]
            label_dict[path] = int(int(label) > diagnosis)

        for dir in datadirs:
            for file in os.listdir(dir):
                self.paths.append(dir+'/'+file)

        self.labels = [label_dict[path[6:]] for path in self.paths]

        neg_weight = np.mean(self.labels)
        self.weights = [neg_weight, 1 - neg_weight]

    def weighted_loss(self, prediction, target):
        weights_npy = np.array([self.weights[int(t[0])] for t in target.data])
        weights_tensor = torch.FloatTensor(weights_npy)
        if self.use_gpu:
            weights_tensor = weights_tensor.cuda()
        loss = F.binary_cross_entropy_with_logits(prediction, target, weight=Variable(weights_tensor))
        return loss

      
    # Data augmentation section
    # can go through each cases, looking at the histogram of 3T vs 1.5T (naive distribution of contrast data?)
    def __getitem__(self, index):
        path = self.paths[index]
        with open(path, 'rb') as file_handler: # Must use 'rb' as the data is binary
            vol = pickle.load(file_handler).astype(np.int32)

        print('vol shape', vol.shape)
        print('vol sum', np.sum(vol))
            
        # crop middle
        pad = int((vol.shape[2] - INPUT_DIM)/2)
        vol = vol[:,pad:-pad,pad:-pad]
        print('vol0 shape', vol.shape)
        
        # standardize
        vol = (vol - np.min(vol)) / (np.max(vol) - np.min(vol)) * MAX_PIXEL_VAL

        # normalize
        vol = (vol - MEAN) / STDDEV
        print('vol1', vol.shape)

        #print('vol shape', vol.shape)

        # convert to RGB
        vol = np.stack((vol,)*3, axis=1)
        print('vol2', vol.shape)

        #new_vol = self.transforms(vol).float()
        #print('new_vol', new_vol.shape)
        
        #if self.transformbool:
        #  vol = np.mean(self.transforms(vol).float(), axis=(0,1))

        #vol = np.stack((vol,)*3, axis=1)
        #print('vol2', vol.shape)

        vol_tensor = torch.FloatTensor(vol)
        label_tensor = torch.FloatTensor([self.labels[index]])
        print('vol tensor shape', vol_tensor.shape)
        print('label_tensor shape', label_tensor.shape)
        return vol_tensor, label_tensor

    def __len__(self):
        return len(self.paths)

def load_data(self, diagnosis, use_gpu=True):
    train_dirs = ['vol08','vol04','vol03','vol09','vol06','vol07']
    valid_dirs = ['vol10','vol05']
    test_dirs = ['vol01','vol02']
    
    #train_dataset = Dataset(train_dirs, diagnosis, use_gpu)
    train_dataset = Dataset(train_dirs, diagnosis, use_gpu)
    valid_dataset = Dataset(valid_dirs, diagnosis, use_gpu)
    test_dataset = Dataset(test_dirs, diagnosis, use_gpu)

    train_loader = data.DataLoader(train_dataset, batch_size=1, num_workers=8, shuffle=True)
    valid_loader = data.DataLoader(valid_dataset, batch_size=1, num_workers=8, shuffle=False)
    test_loader = data.DataLoader(test_dataset, batch_size=1, num_workers=8, shuffle=False)

    return train_loader, valid_loader, test_loader


In [0]:

# evaluate.py

import argparse
import matplotlib.pyplot as plt
import os
import numpy as np
import torch

from sklearn import metrics
from torch.autograd import Variable

#from loader import load_data
#from model import MRNet

def get_parser():
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_path', type=str, required=True)
    parser.add_argument('--split', type=str, required=True)
    parser.add_argument('--diagnosis', type=int, required=True)
    parser.add_argument('--gpu', action='store_true')
    return parser

def run_model(model, loader, train=False, optimizer=None):
    preds = []
    labels = []

    if train:
        model.train()
    else:
        model.eval()

    total_loss = 0.
    num_batches = 0

    for batch in loader:
        if train:
            optimizer.zero_grad()

        vol, label = batch
        if loader.dataset.use_gpu:
            vol = vol.cuda()
            label = label.cuda()
        vol = Variable(vol)
        label = Variable(label)

        logit = model.forward(vol)

        loss = loader.dataset.weighted_loss(logit, label)
        total_loss += loss.item()

        pred = torch.sigmoid(logit)
        pred_npy = pred.data.cpu().numpy()[0][0]
        label_npy = label.data.cpu().numpy()[0][0]

        preds.append(pred_npy)
        labels.append(label_npy)

        if train:
            loss.backward()
            optimizer.step()
        num_batches += 1

    avg_loss = total_loss / num_batches

    fpr, tpr, threshold = metrics.roc_curve(labels, preds)
    auc = metrics.auc(fpr, tpr)

    return avg_loss, auc, preds, labels

def evaluate(split, model_path, diagnosis, use_gpu):
    train_loader, valid_loader, test_loader = load_data(diagnosis, use_gpu)

    model = MRNet()
    state_dict = torch.load(model_path, map_location=(None if use_gpu else 'cpu'))
    model.load_state_dict(state_dict)

    if use_gpu:
        model = model.cuda()

    if split == 'train':
        loader = train_loader
    elif split == 'valid':
        loader = valid_loader
    elif split == 'test':
        loader = test_loader
    else:
        raise ValueError("split must be 'train', 'valid', or 'test'")

    loss, auc, preds, labels = run_model(model, loader)

    print(f'{split} loss: {loss:0.4f}')
    print(f'{split} AUC: {auc:0.4f}')

    return preds, labels

#if __name__ == '__main__':
#    args = get_parser().parse_args()
#   evaluate(args.split, args.model_path, args.diagnosis, args.gpu)

In [0]:
# train.py

import argparse
import json
import numpy as np
import os
import torch

from datetime import datetime
from pathlib import Path
from sklearn import metrics

#from evaluate import run_model
#from loader import load_data
#from model import MRNet

def train(rundir, diagnosis, epochs, learning_rate, use_gpu):
    val_auc_array = list()
    train_auc_array = list()
    train_loader, valid_loader, test_loader = load_data(diagnosis, use_gpu)
    
    model = MRNet()
    
    if use_gpu:
        model = model.cuda()

    # modify the code in this
    optimizer = torch.optim.Adam(model.parameters(), learning_rate, weight_decay=.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=.3, threshold=1e-4)

    best_val_loss = float('inf')

    start_time = datetime.now()

    for epoch in range(epochs):
        change = datetime.now() - start_time
        print('starting epoch {}. time passed: {}'.format(epoch+1, str(change)))
        
        train_loss, train_auc, _, _ = run_model(model, train_loader, train=True, optimizer=optimizer)
        print(f'train loss: {train_loss:0.4f}')
        print(f'train AUC: {train_auc:0.4f}')

        val_loss, val_auc, _, _ = run_model(model, valid_loader)
        print(f'valid loss: {val_loss:0.4f}')
        print(f'valid AUC: {val_auc:0.4f}')
        val_auc_array.append(val_auc)
        train_auc_array.append(train_auc)
        
        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss

            file_name = f'val{val_loss:0.4f}_train{train_loss:0.4f}_epoch{epoch+1}'
            save_path = Path(rundir) / file_name
            # dont need to save stuff for now, model is too shitty
            #torch.save(model.state_dict(), save_path)
            
    return val_auc_array, train_auc_array

def get_parser():
    parser = argparse.ArgumentParser()
    parser.add_argument('--rundir', type=str, required=True)
    parser.add_argument('--diagnosis', type=int, required=True)
    parser.add_argument('--seed', default=42, type=int)
    parser.add_argument('--gpu', action='store_true')
    parser.add_argument('--learning_rate', default=1e-05, type=float)
    parser.add_argument('--weight_decay', default=0.01, type=float)
    parser.add_argument('--epochs', default=50, type=int)
    parser.add_argument('--max_patience', default=5, type=int)
    parser.add_argument('--factor', default=0.3, type=float)
    return parser

#if __name__ == '__main__':
#    args = get_parser().parse_args()
    
#    np.random.seed(args.seed)
#    torch.manual_seed(args.seed)
#    if args.gpu:
#        torch.cuda.manual_seed_all(args.seed)

#    os.makedirs(args.rundir, exist_ok=True)
    
#    with open(Path(args.rundir) / 'args.json', 'w') as out:
#        json.dump(vars(args), out, indent=4)

#    train(args.rundir, args.diagnosis, args.epochs, args.learning_rate, args.gpu)


In [0]:
 import os
%cd "/content/gdrive/My Drive/thesis/Data/vols"

#data path
train_path = "/content/gdrive/My Drive/thesis/Data/train"
data_path = "/content/gdrive/My Drive/thesis/Data/vols"
#train_meniscus_csv_path = data_path + '/train-meniscus.csv'

gpu = True
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

#learningrate = 5e-05
epochs = 20
diagnosis = 0
rundir = data_path

if gpu:
  torch.cuda.manual_seed_all(seed)

varray1, tarray1 = train(rundir, diagnosis, epochs, 2e-05, gpu)
varray2, tarray2 = train(rundir, diagnosis, epochs, 4e-05, gpu)
varray3, tarray3 = train(rundir, diagnosis, epochs, 5e-05, gpu)
varray4, tarray4 = train(rundir, diagnosis, epochs, 6e-05, gpu)
varray5, tarray5 = train(rundir, diagnosis, epochs, 8e-05, gpu)

def display_line(x_length, lr1, y1, lr2, y2, lr3, y3, lr4, y4, lr5, y5, title, xlabel, ylabel):
  plt.figure(0)
  # see if we can set axis later
  #ax = plt.axis()
  #ax.set(xlim = (np.min(x),np.max(x)), option='tight')
  plt.title(title)
  plt.plot(np.arange(x_length), y1, label=str(lr1))
  plt.plot(np.arange(x_length), y2, label=str(lr2))
  plt.plot(np.arange(x_length), y3, label=str(lr3))
  plt.plot(np.arange(x_length), y4, label=str(lr4))
  plt.plot(np.arange(x_length), y5, label=str(lr5))
  plt.legend()
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  plt.show()
  return

display_line(epochs, 2e-05, varray1, 4e-05, varray2, 5e-05, varray3, 6e-05, varray4, 8e-05, varray5, "Trained AlexNet Val AUC over epochs diagnosis = " + str(diagnosis), "epoch", "Validation AUC")
display_line(epochs, 2e-05, tarray1, 4e-05, tarray2, 5e-05, tarray3, 6e-05, tarray4, 8e-05, tarray5, "Trained AlexNet Train AUC over epochs diagnosis = " + str(diagnosis), "epoch", "Training AUC")

/content/gdrive/My Drive/thesis/Data/vols
['vol08', 'vol04', 'vol03', 'vol09', 'vol06', 'vol07']
['vol10', 'vol05']
['vol01', 'vol02']
starting epoch 1. time passed: 0:00:00.000015
vol shape (30, 320, 320)
vol shape (34, 320, 320)
vol sum 922462271
vol0 shape (34, 224, 224)
vol sum 857956661
vol0 shape (30, 224, 224)
vol shape (32, 320, 320)
vol sum 881650906
vol shape (30, 320, 320)
vol0 shape (32, 224, 224)
vol shape (32, 320, 320)
vol shape (30, 320, 320)
vol shape (32, 320, 320)
vol shape (30, 320, 320)
vol sum 928543369
vol0 shape (30, 224, 224)
vol sum 898855983
vol sum 841726209
vol sum 928735787
vol sum 873606930
vol0 shape (32, 224, 224)
vol0 shape (30, 224, 224)
vol0 shape (30, 224, 224)
vol0 shape (32, 224, 224)
vol1 (30, 224, 224)
vol1 (34, 224, 224)
vol1 (32, 224, 224)
vol2 (34, 3, 224, 224)
vol2 (30, 3, 224, 224)
vol1 (30, 224, 224)
vol1 (32, 224, 224)
vol1 (30, 224, 224)
vol1 (32, 224, 224)
vol1 (30, 224, 224)
vol tensor shape torch.Size([30, 3, 224, 224])
label_tensor s

In [0]:

#evaluate('test', 'val0.0175_train0.0139_epoch50', 1, True)

In [0]:

def display1_line(x_length, lr1, y1, lr2, y2, lr3, y3, lr4, y4, lr5, y5, title, xlabel, ylabel):
  plt.figure(0)
  # see if we can set axis later
  #ax = plt.axis()
  #ax.set(xlim = (np.min(x),np.max(x)), option='tight')
  plt.title(title)
  plt.plot(np.arange(x_length), y1, label=str(lr1))
  plt.plot(np.arange(x_length), y2, label=str(lr2))
  plt.plot(np.arange(x_length), y3, label=str(lr3))
  plt.plot(np.arange(x_length), y4, label=str(lr4))
  plt.plot(np.arange(x_length), y5, label=str(lr5))
  plt.legend()
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  plt.show()
  return

display1_line(epochs, 5e-06, varray1, 1e-05, varray2, 5e-05, varray3, 1e-04, varray4, 1e-03, varray5, "AUC loss over epochs diagnosis = " + str(diagnosis), "epoch", "Validation AUC")

In [0]:
# notes from ben
# get rid of double standardization, dont have RGB channels all have same information, make AlexNet take 1 channel instead of 3. 
# try to dissect images into two camps, 3.0T and 1.5T.
# Then we can try to change into DenseNets. 